In [100]:
import argparse
import logging
import os
import pdb
import random
import numpy as np
from cnf import CNF
import pysat
from os import listdir
from pysat.formula import CNF
from pysat.solvers import Lingeling,Minisat22, Glucose4
import pickle
from pysat.examples.rc2 import RC2

## WALKSAT CODE

In [101]:
def parse(filename):
    clauses = []
    count = 0
    for line in open(filename):

        if line[0] == 'c':
            continue
        if line[0] == 'p':
            n_vars = int(line.split()[2])
            lit_clause = [[] for _ in range(n_vars * 2 + 1)]
            continue

        clause = []
        for literal in line[:-2].split():
            literal = int(literal)
            clause.append(literal)
            lit_clause[literal].append(count)
        clauses.append(clause)
        count += 1
    return clauses, n_vars, lit_clause


def get_random_interpretation(n_vars):
    return [i if random.random() < 0.5 else -i for i in range(n_vars + 1)]


def get_true_sat_lit(clauses, interpretation):
    true_sat_lit = [0 for _ in clauses]
    for index, clause in enumerate(clauses):
        for lit in clause:
            if interpretation[abs(lit)] == lit:
                true_sat_lit[index] += 1
    return true_sat_lit


def update_tsl(literal_to_flip, true_sat_lit, lit_clause):
    for clause_index in lit_clause[literal_to_flip]:
        true_sat_lit[clause_index] += 1
    for clause_index in lit_clause[-literal_to_flip]:
        true_sat_lit[clause_index] -= 1


def compute_broken(clause, true_sat_lit, lit_clause, omega=0.4):
    break_min = 999999999
    best_literals = []
    for literal in clause:

        break_score = 0

        for clause_index in lit_clause[-literal]:
            if true_sat_lit[clause_index] == 1:
                break_score += 1

        if break_score < break_min:
            break_min = break_score
            best_literals = [literal]
        elif break_score == break_min:
            best_literals.append(literal)

    if break_min != 0 and random.random() < omega:
        best_literals = clause

    return random.choice(best_literals)


def run_sat(clauses, n_vars, lit_clause, max_flips_proportion=4, max_tries=500):
    max_flips = n_vars * max_flips_proportion
    tries = 0
    num_unsat = len(clauses)
    while (tries <= max_tries):
        interpretation = get_random_interpretation(n_vars)
        true_sat_lit = get_true_sat_lit(clauses, interpretation)
        for _ in range(max_flips):

            unsatisfied_clauses_index = [index for index, true_lit in enumerate(true_sat_lit) if
                                         not true_lit]

            if not unsatisfied_clauses_index:
                num_unsat = 0 
                return interpretation, num_unsat

            clause_index = random.choice(unsatisfied_clauses_index)
            unsatisfied_clause = clauses[clause_index]

            lit_to_flip = compute_broken(unsatisfied_clause, true_sat_lit, lit_clause)

            update_tsl(lit_to_flip, true_sat_lit, lit_clause)

            interpretation[abs(lit_to_flip)] *= -1
            if len(unsatisfied_clauses_index) <= num_unsat:
                num_unsat = len(unsatisfied_clauses_index)
        tries+=1 
    return interpretation, num_unsat


## Check satisfiability of instances in directory with pysat/glucose

In [89]:
file_scores = {}
dirname = 'dimacs_400_random/' ## ENTER DIRECTORY NAME WITH DIMACS FILES HERE
num_files = len(listdir(dirname))
count_unsat = 0
unsat_list = []
unsat_example = None 
for fl in listdir(dirname):
    my_formula = CNF(from_file=dirname+fl)
   # print(fl)
    with Glucose4(bootstrap_with=my_formula.clauses, with_proof=True) as l:
        is_sat = (l.solve())
        if  not is_sat:
            unsat_list += [fl.split('_')[1]]
            count_unsat+=1
            unsat_example = my_formula
        file_scores[fl] = [is_sat*1.]

## Run walksat on instances in directory

In [95]:
dirname = 'dimacs_400_random/'
num_files = len(listdir(dirname))
count_unsat = 0
#unsat_list = []
num_unsatclauses_perinstance = []
unsat_example = None 
instance_count = 0
for fl in listdir(dirname):
    clauses, n_vars, lit_clause = parse(dirname+fl)
    solution, num_unsat = run_sat(clauses, n_vars, lit_clause, max_tries=100)
    num_unsatclauses_perinstance += [num_unsat]
    instance_count += 1 
    file_scores[fl] +=[num_unsat]
    print("Current instance: ", instance_count, "Num unsat clauses instance: ", num_unsat)

Current instance:  1 Num unsat clauses instance:  0
Current instance:  2 Num unsat clauses instance:  0
Current instance:  3 Num unsat clauses instance:  0
Current instance:  4 Num unsat clauses instance:  1
Current instance:  5 Num unsat clauses instance:  0
Current instance:  6 Num unsat clauses instance:  0
Current instance:  7 Num unsat clauses instance:  0
Current instance:  8 Num unsat clauses instance:  0
Current instance:  9 Num unsat clauses instance:  0
Current instance:  10 Num unsat clauses instance:  1
Current instance:  11 Num unsat clauses instance:  0
Current instance:  12 Num unsat clauses instance:  1
Current instance:  13 Num unsat clauses instance:  1
Current instance:  14 Num unsat clauses instance:  1
Current instance:  15 Num unsat clauses instance:  0
Current instance:  16 Num unsat clauses instance:  1
Current instance:  17 Num unsat clauses instance:  0
Current instance:  18 Num unsat clauses instance:  0
Current instance:  19 Num unsat clauses instance:  1
Cu

## Sanity check: check number of unsat clauses after running walksat. This number should obviously not be 0 for unsat instances.

In [98]:
for key in file_scores.keys():
    if file_scores[key][0]==0:
        #print(file_scores[key][1])
        if file_scores[key][1]==file_scores[key][0]:
            print("Error: Walksat finds somehow a 0 unsat clause assignment on an unsat formula.")


## Print walksat results

In [105]:
print("Average number of unsat clauses by Walksat on the test instances: ", sum(num_unsatclauses_perinstance)/len(num_unsatclauses_perinstance))

Average number of unsat clauses by Walksat on the test instances:  0.146
